# Парсинг веб-страниц

*В этом задании вы потренеруетесь парсить веб-страницы. Это умение пригодится вам, когда возникнет потребность самостоятельно собрать выборку для построения модели, будь то тексты или какая-то другая информация с общедоступных веб-страниц. В рамках данного проекта это случится уже в задании следующей недели.*

## Загрузка библиотек

In [1]:
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool
from functools import reduce

## Получение количества страниц

In [2]:
URL = "https://www.metacritic.com/browse/games/release-date/available/ps4"
html = requests.get(URL, headers={'User-Agent': 'Mozilla/5.0'}).content
soup = BeautifulSoup(html, 'html.parser')
num_pages = int(soup.find('li', class_='page last_page').find('a', class_='page_num').text)

In [3]:
print(f"Количество страниц - {num_pages}")

Количество страниц - 22


## Парсинг страниц

In [4]:
def get_games_list(url):
    html = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).content
    soup = BeautifulSoup(html, 'html.parser')
    games = soup.find('ol', class_='list_products list_product_condensed')
    titles = games.findAll('div', class_='basic_stat product_title')
    return [title.find('a').text.strip() for title in titles]

In [5]:
with Pool(10) as p:
    url_list = [URL + "/date?page=" + str(page) for page in range(num_pages)]
    map_results = p.map(get_games_list, url_list)
    games_list = sorted(reduce(lambda x, y: x + y, map_results))

In [6]:
games_list[:10]

['#KILLALLZOMBIES',
 "'n Verlore Verstand",
 '.hack//G.U. Last Recode',
 '10 Second Ninja X',
 '1000 Top Rated',
 '1001 Spikes',
 '100ft Robot Golf',
 '101 Ways to Die',
 '11-11: Memories Retold',
 '11-11: Memories Retold - WarChild Charity']

In [7]:
with open('ps4_games.txt', 'w') as f:
    f.write('\n'.join(games_list))